In [1]:
#step one, get the top 100
import pandas as pd
from bs4 import BeautifulSoup
import requests

import os
# shazamapi needds ffmpeg , but it doesn't really have to be imported anywhere : import ffmpeg


In [2]:
#Initialize SpotiPy with user credentias
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config



In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,  client_secret= config.client_secret))
  

In [4]:
# local libraries
import shazaam

In [5]:

shazaam.shazaam(debug=True)


recording for 5 seconds
Starting: Play music now!
finished
that was "Enemy (From the series "Arcane League of Legends")" from Imagine Dragons, JID & League of Legends


('Enemy (From the series "Arcane League of Legends")',
 'Imagine Dragons, JID & League of Legends')

In [6]:
def soundex(name):


    """
    The Soundex algorithm assigns a 1-letter + 3-digit code to strings,
    the intention being that strings pronounced the same but spelled
    differently have identical encodings; words pronounced similarly
    should have similar encodings.
    """

    soundexcoding = [' ', ' ', ' ', ' ']
    soundexcodingindex = 1

    #           ABCDEFGHIJKLMNOPQRSTUVWXYZ
    mappings = "01230120022455012623010202"

    soundexcoding[0] = name[0].upper()

    for i in range(1, len(name)):

         c = ord(name[i].upper()) - 65

         if c >= 0 and c <= 25:

             if mappings[c] != '0':

                 if mappings[c] != soundexcoding[soundexcodingindex-1]:

                     soundexcoding[soundexcodingindex] = mappings[c]
                     soundexcodingindex += 1

                 if soundexcodingindex > 3:

                     break

    if soundexcodingindex <= 3:
        while(soundexcodingindex <= 3):
            soundexcoding[soundexcodingindex] = '0'
            soundexcodingindex += 1

    return ''.join(soundexcoding)

In [7]:
def is_recognizable():
    return len(shazam_info[1]["matches"])>0

def get_song_info():
    return shazam_info[1]["track"]["title"], shazam_info[1]["track"]["subtitle"]

In [8]:
URL = 'https://www.billboard.com/charts/hot-100/'
Top100Website = requests.get(URL)
result = BeautifulSoup(Top100Website.content, 'html.parser')

In [9]:
result.find("span", id="count_texttitle-of-a-story")

In [10]:
line_number = 0
place_number = 0
Song = "";
Artists = "";
Top_100 = pd.DataFrame(columns=['place','song','artists', 'song_soundex'])
print(str(Top_100))

for lines in result.find_all("li", {"o-chart-results-list__item"}):
    line_number += 1

 
    if((line_number-4)%14==0):
        place_number += 1
        internal_line = 1
        for current_line in lines.get_text().splitlines():

            if(current_line.strip()!=""):
                if(internal_line==1):
                    Song = current_line.strip()
                else:
                    Artists = current_line.strip()

                    Top_100.loc[place_number -1] = [place_number, Song, Artists, soundex(str(Song))]
 
                internal_line += 1

Empty DataFrame
Columns: [place, song, artists, song_soundex]
Index: []


In [11]:
#temporary
Top_100

,place,song,artists,song_soundex
0,1,As It Was,Harry Styles,A232
1,2,First Class,Jack Harlow,F623
2,3,Heat Waves,Glass Animals,H312
3,4,Big Energy,Latto,B256
4,5,Enemy,Imagine Dragons X JID,E500
...,...,...,...,...
95,96,P Power,Gunna Featuring Drake,P160
96,97,Money So Big,Yeat,M521
97,98,Blick Blick!,Coi Leray & Nicki Minaj,B421
98,99,Fall In Love,Bailey Zimmerman,F454


In [12]:
import config
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,  client_secret= config.client_secret))


In [13]:
recommend_me_something_for_this = input("Please enter a song to base a recommendation on ")

Please enter a song to base a recommendation on cry


In [ ]:
if(recommend_me_something_for_this=="shazaam"):
    #use the local shazaam library to identify the song currently playing
    recorded = shazaam.shazaam()

    #returns false if nothing was found
    if(recorded != False):
        #and a tuple of song and artist if it worked
        recorded_song, recorded_artist = recorded 
        print(recorded_song + " " + recorded_artist)
    
        #replace the input with the actual song
        recommend_me_something_for_this = recorded_song
  
Number_Of_Songs_Found = len(Top_100[Top_100['song_soundex'] == soundex(recommend_me_something_for_this.strip())])
if(Number_Of_Songs_Found>0):
    #find a new song
    print("Let me recommend you:")
    recommendation_dataframe = Top_100.sample(n=1)
    print(str(recommendation_dataframe ))
else:
    if(len(recommend_me_something_for_this.strip())>0):
        search_results = sp.search(q=recommend_me_something_for_this.strip(),limit=50,market="GB")
        if(len(search_results["tracks"])>1):
            print("Please select a number regarding the many titles of this name")
            for i in range(0,len(search_results["tracks"]["items"])):

                print("["+str(i+1)+"] " + str(search_results["tracks"]["items"][i]["name"]))
                print("by")
                for j in range(0,len(search_results["tracks"]["items"][i]["artists"])):

                    print(str(search_results["tracks"]["items"][i]["artists"][j]["name"]))
                print(" ")
           # print(str(search_results["tracks"]["items"][i]["name"]))        
        else:
            if(len(search_results)==1):
                print(str(search_results["tracks"]["items"][0]["name"]))
                print("by")
                for j in range(0,len(search_results["tracks"]["items"][0]["artists"])):
                    print("  " + str(search_results["tracks"]["items"][0]["artists"][j]["name"]))
        input("which number do you choose?")
    print("“Unfortunately, the song is not in the hot list!")    

Please select a number regarding the many titles of this name
[1] Cry Me a River
by
Justin Timberlake
 
[2] That Should Be Me
by
Justin Bieber
 
[3] Crying Lightning
by
Arctic Monkeys
 
[4] Laugh Now Cry Later (feat. Lil Durk)
by
Drake
Lil Durk
 
[5] Cry to Me
by
Solomon Burke
 
[6] Cry Your Heart Out
by
Adele
 
[7] Big Girls Don't Cry (Personal)
by
Fergie
 
[8] Cry for You
by
September
 
[9] Crying in the Club
by
Camila Cabello
 
[10] Boys Don't Cry
by
The Cure
 
[11] Crystalised
by
The xx
 
[12] Stop Crying Your Heart Out
by
Oasis
 
[13] Cry to Me
by
Kilotile
 
[14] Cry for You - UK Radio Edit
by
September
 
[15] no tears left to cry
by
Ariana Grande
 
[16] Cry
by
Cigarettes After Sex
 
[17] Cry to Me
by
Solomon Burke
 
[18] Bored
by
Billie Eilish
 
[19] Cry Baby
by
Melanie Martinez
 
[20] Cry Baby
by
The Neighbourhood
 
[21] Cry (Just A Little) - Radio Edit
by
Bingo Players
 
[22] changes
by
XXXTENTACION
 
[23] Cry Me a River
by
Michael Bublé
 
[24] Body Groove (feat. Crystxl King)


In [25]:
        search_results = sp.search(q="enemy",limit=50,market="GB")



In [36]:
search_results["tracks"]["items"]



[{'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/53XhwfbYqKCa1cC15pYq2q'},
     'href': 'https://api.spotify.com/v1/artists/53XhwfbYqKCa1cC15pYq2q',
     'id': '53XhwfbYqKCa1cC15pYq2q',
     'name': 'Imagine Dragons',
     'type': 'artist',
     'uri': 'spotify:artist:53XhwfbYqKCa1cC15pYq2q'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/6U3ybJ9UHNKEdsH7ktGBZ7'},
     'href': 'https://api.spotify.com/v1/artists/6U3ybJ9UHNKEdsH7ktGBZ7',
     'id': '6U3ybJ9UHNKEdsH7ktGBZ7',
     'name': 'JID',
     'type': 'artist',
     'uri': 'spotify:artist:6U3ybJ9UHNKEdsH7ktGBZ7'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/57nPqD7z62gDdq37US9XJR'},
     'href': 'https://api.spotify.com/v1/artists/57nPqD7z62gDdq37US9XJR',
     'id': '57nPqD7z62gDdq37US9XJR',
     'name': 'Arcane',
     'type': 'artist',
     'uri': 'spotify:artist:57nPqD7z62gDdq37US9XJR'}],
   'external_urls': {'spotify': 'h

Enemy (with JID) - from the series Arcane League of Legends
Imagine Dragons
JID
Arcane
League of Legends
Enemy - From the series Arcane League of Legends
Imagine Dragons
Arcane
League of Legends
Enemy (with JID) - from the series Arcane League of Legends
Imagine Dragons
JID
Arcane
League of Legends
Enemy feat. J.I.D. (from the series Arcane League of Legends)
Imagine Dragons
JID
Enemy
Tommee Profitt
Beacon Light
Sam Tinnesz
My Own Worst Enemy
Lit
Enemy - Epic Version (from Arcane League of Legends)
Samuel Kim
ENEMY
slowthai
Harder Than You Think
Public Enemy
Enemy
Zack Knight
Know Your Enemy
Green Day
Enemy
Arc North
Meric Again
Fallen Roses
We'll Live and Die in These Towns
The Enemy
Enemy (Arcane opening theme)
Jyc Row
Shelley Harland
Enemy (from "Arcane League of Legends")
AmaLee
Time Is the Enemy
Quantic
Enemy (Arcane League of Legends) - Epic Version
Krutikov Music
Enemy
Our Last Night
The Enemy
Andrew Belle
Believer
Imagine Dragons
Enemy
Sevendust
Know Your Enemy
Rage Against The

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=enemy&type=track&market=GB&offset=0&limit=50',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/53XhwfbYqKCa1cC15pYq2q'},
       'href': 'https://api.spotify.com/v1/artists/53XhwfbYqKCa1cC15pYq2q',
       'id': '53XhwfbYqKCa1cC15pYq2q',
       'name': 'Imagine Dragons',
       'type': 'artist',
       'uri': 'spotify:artist:53XhwfbYqKCa1cC15pYq2q'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/6U3ybJ9UHNKEdsH7ktGBZ7'},
       'href': 'https://api.spotify.com/v1/artists/6U3ybJ9UHNKEdsH7ktGBZ7',
       'id': '6U3ybJ9UHNKEdsH7ktGBZ7',
       'name': 'JID',
       'type': 'artist',
       'uri': 'spotify:artist:6U3ybJ9UHNKEdsH7ktGBZ7'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/57nPqD7z62gDdq37US9XJR'},
       'href': 'https://api.spotify.com/v1/artists/57nPqD7z62gDdq37US9XJR',
       'id': '57nPqD7z62g